# Topics In Bioinformatics - Wk4 Assignment - Dr. Jody Hey 

## Madison Dautle
## Created on Sept. 22, 2025
## Last updated on Sept. 22, 2025

In [17]:
import pandas as pd 
import time
import os

In [18]:
#Read in the individuals list and show population options
individuals_df = pd.read_csv('./integrated_call_samples_v3.20130502.ALL.panel', sep='\t', header = 0, usecols = [0,1,2]) 

#unique_superpop = individuals_df['super_pop'].unique()
unique_pop = individuals_df['pop'].unique()

#print('Super Population Choices: \n') 
#print(unique_superpop) 
print('Population Choices: \n') 
print(unique_pop)

Population Choices: 

['GBR' 'FIN' 'CHS' 'PUR' 'CDX' 'CLM' 'IBS' 'PEL' 'PJL' 'KHV' 'ACB' 'GWD'
 'ESN' 'BEB' 'MSL' 'STU' 'ITU' 'CEU' 'YRI' 'CHB' 'JPT' 'LWK' 'ASW' 'MXL'
 'TSI' 'GIH']


In [19]:
#Choose (a) population(s) for analysis 
selc_pop = ['YRI','GBR','JPT'] 

In [20]:
#Set up output to pdf
from reportlab.lib import colors
from reportlab.lib.pagesizes import letter, portrait
from reportlab.platypus import SimpleDocTemplate, Table, TableStyle, Paragraph, Spacer, PageBreak
from reportlab.lib.styles import getSampleStyleSheet

outFile = 'summary.pdf' 
pdf = SimpleDocTemplate(outFile, pagesize=portrait(letter))
elements = []
styles = getSampleStyleSheet() 
header_style = styles["Heading2"] 

def write_df_to_pdf(df): 
    data = [df.columns.to_list()] + df.values.tolist()
    table = Table(data) 
    table.setStyle(TableStyle([
        ("BACKGROUND", (0, 0), (-1, 0), colors.grey),
        ("TEXTCOLOR", (0, 0), (-1, 0), colors.whitesmoke),
        ("ALIGN", (0, 0), (-1, -1), "CENTER"),
        ("FONTNAME", (0, 0), (-1, 0), "Helvetica-Bold"),
        ("BOTTOMPADDING", (0, 0), (-1, 0), 12), 
        ("BACKGROUND", (0, 1), (-1, -1), colors.beige), 
        ("GRID", (0, 0), (-1, -1), 1, colors.black), 
    ]))
    return table


In [21]:
if not os.path.exists('./madison.vcf'):
    print('Removing header lines from VCF file, so it can be read in with the column names intact') 
    command = "grep -v '^##' ./ALL.chr22.phase3_shapeit2_mvncall_integrated_v5a.20130502.genotypes.vcf | sed 's/^#//g' > ./madison.vcf"
    os.system(command)

all_results = pd.DataFrame(columns=['POS', 'REF', 'ALT','Observed_heterozygosity','Expected_heterozygosity','obs-exp','Population']) 
summary_table = pd.DataFrame(index = selc_pop, columns=['Avg_obs_hetero','Avg_exp_hetero','Avg_diff'])
for pop in selc_pop: 
    print(f'Running analysis for {pop} population')
    start_tot_time = time.time()
    start_time = time.time()
    keepcols = ['POS','REF','ALT']
    pop_samples = individuals_df.loc[individuals_df['pop'] == pop, 'sample'].tolist()
    keepcols = keepcols+pop_samples
    #print(keepcols)
    vcf = pd.read_csv('./madison.vcf', sep = '\t', usecols = keepcols, header = 0)
    end_time = time.time() 
    print(f'Total time to read in file: {end_time-start_time}')
    #print(vcf.columns)

    
    #Remove indels 
    start_time = time.time()
    vcf = vcf[(vcf['REF'].str.len() == 1) & (vcf['ALT'].str.len() == 1)] 
    end_time = time.time() 
    print(f'Total time to remove indel postions from file: {end_time-start_time} sec')


    #Remove SNPs that have more than one variant 
    start_time = time.time()
    vcf = vcf[vcf[pop_samples].apply(lambda row: row.isin(['0|0','0|1','1|0','1|1']).all(), axis=1)]
    end_time = time.time() 
    print(f'Total time to remove indel postions from file: {end_time-start_time} sec')
    #print(vcf.head()) 


    #Remove positions where there are no SNPs 
    start_time = time.time()
    mask = ~(vcf[pop_samples] == '0|0').all(axis=1)
    vcf = vcf[mask] 
    end_time = time.time() 
    print(f'Total time to remove non SNP postions from file: {end_time-start_time} sec')
    #print(vcf.head()) 


    #Count heterozygotes & calculate obsrved heterozygosity 
    start_time = time.time()
    vcf['num_hets']  = vcf[pop_samples].apply(lambda row: ((row == '0|1') | (row == '1|0')).sum(), axis=1)
    vcf['Observed_heterozygosity'] = vcf['num_hets']/len(pop_samples) 
    end_time = time.time() 
    print(f'Total time to count observed heterozygtes @ each SNP & calculate heterozygosity: {end_time-start_time} sec')
    #print(vcf[['num_hets','Observed_heterozygosity']])


    #Calculate allelic frequencies & expected heterozygosity 
    start_time = time.time()
    genotype_data = vcf[pop_samples] 
    vcf['raw_num_p'] = genotype_data.apply(lambda row: row.str.replace('|','').str.count('0').sum(), axis=1)
    vcf['raw_num_q'] = genotype_data.apply(lambda row: row.str.replace('|','').str.count('1').sum(), axis=1)
    #vcf['test'] = len(pop_samples) - ((vcf['raw_num_p'] + vcf['raw_num_q'])/2) # all columns should be equal to 0
    #print(vcf[['test']])
    vcf['p'] = vcf['raw_num_p']/(vcf['raw_num_p'] + vcf['raw_num_q'])
    vcf['q'] = vcf['raw_num_q']/(vcf['raw_num_p'] + vcf['raw_num_q'])
    #vcf['test1'] = vcf['p'] + vcf['q'] 
    #print(vcf[['test1']]) #all columns should be equal to 1
    vcf['Expected_heterozygosity'] = 2*vcf['p']*vcf['q']
    end_time = time.time() 
    print(f'Total time to count allelic frequencies @ each SNP & calculate expected heterozygosity: {end_time-start_time} sec')
    #print(vcf[['Observed_heterozygosity','Expected_heterozygosity']])


    #Calculate Obs-Expected heterozygosity 
    vcf['obs-exp'] = vcf['Observed_heterozygosity'] - vcf['Expected_heterozygosity'] 
    print(vcf[['Observed_heterozygosity','Expected_heterozygosity','obs-exp']])


    #Calculate average observed expected, & difference for the population and write to summary table 
    summary_table.loc[pop, 'Avg_obs_hetero'] = vcf['Observed_heterozygosity'].mean()
    summary_table.loc[pop, 'Avg_exp_hetero'] = vcf['Expected_heterozygosity'].mean()
    summary_table.loc[pop, 'Avg_diff'] = vcf['obs-exp'].mean()
    summary_table = summary_table.round(5) 
    end_tot_time = time.time() 


    #Add population dataframe to pdf 
    pop_df = vcf[['POS', 'REF', 'ALT','Observed_heterozygosity','Expected_heterozygosity','obs-exp']].copy()
    pop_df = pop_df.round(5)
    elements.append(Paragraph(f"Population: {pop}", header_style))
    elements.append(Spacer(1, 12))
    elements.append(write_df_to_pdf(pop_df))
    elements.append(PageBreak())
    del pop_df
    del vcf
    
    print(f'Total processing time for {pop}: {(end_tot_time - start_tot_time)/60} min') 


Running analysis for YRI population
Total time to read in file: 23.891176223754883
Total time to remove indel postions from file: 1.9350988864898682 sec
Total time to remove indel postions from file: 32.039238691329956 sec
Total time to remove non SNP postions from file: 5.272151231765747 sec
Total time to count observed heterozygtes @ each SNP & calculate heterozygosity: 16.98350429534912 sec
Total time to count allelic frequencies @ each SNP & calculate expected heterozygosity: 58.562153339385986 sec
         Observed_heterozygosity  Expected_heterozygosity   obs-exp
1                       0.092593                 0.088306  0.004287
2                       0.046296                 0.062714 -0.016418
17                      0.037037                 0.036351  0.000686
26                      0.037037                 0.036351  0.000686
54                      0.009259                 0.009216  0.000043
...                          ...                      ...       ...
1103536         

In [22]:
#Add summary table to pdf
start_time = time.time()

elements.append(Paragraph("Summary Table", header_style))
elements.append(Spacer(1, 12))
summary_tbl = write_df_to_pdf(summary_table.reset_index())
elements.append(summary_tbl)
elements.append(PageBreak())

#Write pdf to file
pdf.build(elements)

end_time = time.time()

print(f'Total time taken to write summary of results to pdf: {(end_time-start_time)} sec') 

Total time taken to write summary of results to pdf: 1854.6915848255157 sec
